In [ ]:
# Market Intelligence System - Dependencies Installation
import sys
import subprocess
from pathlib import Path

# Install dependencies for Market Intelligence
req_file = Path("requirements.txt")
extra_packages = [
    "beautifulsoup4>=4.12.0",
    "feedparser>=6.0.10", 
    "tweepy>=4.14.0",
    "sec-edgar-downloader>=5.0.0",
    "fredapi>=0.5.1",
    "lxml>=4.9.3",
    "html5lib>=1.1",
    "loguru>=0.7.0",
]

if req_file.exists():
    cmd = [sys.executable, "-m", "pip", "install", "-r", str(req_file)] + extra_packages
    print("Installing dependencies...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Dependencies installed successfully")
    else:
        print("⚠️ Some packages may already be installed")
else:
    print("⚠️ requirements.txt not found, installing extra packages only")
    cmd = [sys.executable, "-m", "pip", "install"] + extra_packages
    subprocess.run(cmd, check=False)

print(f"Python version: {sys.version}")



In [ ]:
# Environment Setup and Configuration
import os
import sys
from pathlib import Path

# Load environment variables from .env file if it exists
env_path = Path(".env")
if env_path.exists():
    print("Loading environment variables from .env file...")
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue
        key, value = line.split('=', 1)
        os.environ[key.strip()] = value.strip()
    print("✅ Environment variables loaded")
else:
    print("ℹ️ No .env file found - using default configuration")

# Configure S&P 500 analysis scope
# Remove any test limits for full analysis
os.environ.pop("CMI_TICKERS_LIMIT", None)
print("📊 Configured for full S&P 500 analysis")

# Add current directory to Python path for imports
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

# Import the market intelligence module
try:
    import comprehensive_market_intelligence as cmi
    print("✅ Market Intelligence module imported successfully")
except ImportError as e:
    print(f"❌ Error importing module: {e}")
    print("Make sure comprehensive_market_intelligence.py is in the current directory")



In [ ]:
# Market Intelligence Analysis Pipeline
from importlib import reload
from pathlib import Path
from IPython.display import HTML, display
import time

# Reload module to ensure latest changes
reload(cmi)

# Configure the intelligence system
config = cmi.IntelligenceConfig(
    max_workers=8,                    # Parallel processing threads
    min_conviction_score=50.0,        # Minimum confidence threshold
    max_recommendations=12,           # Top recommendations to generate
    output_dir=Path("market_intelligence")  # Output directory
)

print("🚀 Initializing Market Intelligence Engine...")
engine = cmi.ComprehensiveMarketIntelligence(config)

print("📊 Starting comprehensive market analysis...")
start_time = time.time()

# Generate the comprehensive report
report = engine.generate_comprehensive_report()

end_time = time.time()
analysis_time = end_time - start_time

# Display summary results
print("\n" + "="*60)
print("📈 MARKET INTELLIGENCE ANALYSIS COMPLETE")
print("="*60)
print(f"⏱️  Analysis Time: {analysis_time:.1f} seconds")
print(f"📊 Stocks Analyzed: {report['summary']['total_stocks_analyzed']}")
print(f"🎯 Recommendations Generated: {report['summary']['recommendations_generated']}")
print(f"⭐ Strong Buys: {report['summary']['strong_buys']}")
print(f"📈 Buy Recommendations: {report['summary']['buys']}")
print(f"🔍 Data Sources Active: {report['summary']['data_sources_active']}")

# Display top picks
if report.get('top_picks'):
    print("\n🏆 TOP RECOMMENDATIONS:")
    for i, pick in enumerate(report['top_picks'][:5], 1):
        print(f"  {i}. {pick['ticker']} - {pick['recommendation']} "
              f"(Conviction: {pick['conviction_score']}%)")
        print(f"     {pick['company']} | Catalysts: {pick['key_catalysts']}")

# Display HTML dashboard
out_dir = config.output_dir
html_files = sorted(Path(out_dir).glob("dashboard_*.html"))
if html_files:
    latest_html = html_files[-1]
    print(f"\n📊 Rendering dashboard: {latest_html.name}")
    display(HTML(latest_html.read_text()))
else:
    print(f"\n⚠️ No HTML dashboard found in {out_dir}")

print(f"\n📁 All reports saved to: {out_dir}")
print("✅ Market Intelligence Analysis Complete!")



# 📊 Market Intelligence System

## Overview
This notebook demonstrates a comprehensive market intelligence system that integrates multiple data sources to generate stock recommendations:

- **SEC Filings** - Material events and insider trades
- **Economic Calendar** - Fed meetings and economic indicators  
- **Congressional Trades** - STOCK Act disclosures
- **Social Sentiment** - Social media trend analysis
- **Institutional Activity** - Smart money tracking
- **Technical Analysis** - RSI, momentum, volume patterns
- **Fundamental Data** - P/E ratios, market cap analysis

## Usage
1. Run the first cell to install dependencies
2. Run the second cell to configure the environment
3. Run the third cell to execute the analysis pipeline

The system will generate:
- HTML dashboard with visual results
- JSON report with complete data
- CSV file with buy/sell recommendations


In [ ]:
# Additional Analysis and Configuration Examples

# Example 1: Conservative Investment Strategy
print("🔧 Example: Conservative Investment Configuration")
conservative_config = cmi.IntelligenceConfig(
    max_workers=4,
    min_conviction_score=70.0,  # Higher threshold for conservative approach
    max_recommendations=8,      # Fewer, higher-quality picks
    technical_weight=0.20,      # Lower technical weight
    fundamental_weight=0.30,    # Higher fundamental weight
    insider_weight=0.20,        # Higher insider weight
    institutional_weight=0.20,  # Higher institutional weight
    social_weight=0.05,         # Lower social weight
    macro_weight=0.05
)

# Example 2: Growth Investment Strategy  
print("🔧 Example: Growth Investment Configuration")
growth_config = cmi.IntelligenceConfig(
    max_workers=8,
    min_conviction_score=55.0,  # Lower threshold for more opportunities
    max_recommendations=15,     # More picks for diversification
    technical_weight=0.30,      # Higher technical weight
    fundamental_weight=0.15,    # Lower fundamental weight
    insider_weight=0.10,        # Lower insider weight
    institutional_weight=0.25,  # Higher institutional weight
    social_weight=0.15,         # Higher social weight
    macro_weight=0.05
)

print("✅ Configuration examples created")
print("💡 Use these configurations with: engine = cmi.ComprehensiveMarketIntelligence(config)")


In [ ]:
# Data Export and Analysis Tools

import pandas as pd
import json
from pathlib import Path

def export_analysis_data(report, output_dir="analysis_export"):
    """Export analysis data to various formats for further analysis"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # Export recommendations to Excel with multiple sheets
    if report.get('recommendations'):
        df_recs = pd.DataFrame(report['recommendations'])
        excel_path = output_path / "market_analysis.xlsx"
        
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            df_recs.to_excel(writer, sheet_name='Recommendations', index=False)
            
            # Create summary sheet
            summary_data = {
                'Metric': ['Total Stocks Analyzed', 'Recommendations Generated', 
                          'Strong Buys', 'Buy Recommendations', 'Data Sources Active'],
                'Value': [report['summary']['total_stocks_analyzed'],
                         report['summary']['recommendations_generated'],
                         report['summary']['strong_buys'],
                         report['summary']['buys'],
                         report['summary']['data_sources_active']]
            }
            pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
        
        print(f"📊 Analysis exported to: {excel_path}")
    
    # Export raw intelligence data
    intel_path = output_path / "intelligence_data.json"
    with open(intel_path, 'w') as f:
        json.dump(report['market_intelligence'], f, indent=2, default=str)
    
    print(f"📋 Intelligence data exported to: {intel_path}")
    return output_path

# Example usage (uncomment to run):
# export_path = export_analysis_data(report)
# print(f"✅ All data exported to: {export_path}")

print("🔧 Data export functions ready")
print("💡 Use: export_path = export_analysis_data(report) to export data")
